In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install icrawler


In [ ]:
pip install requests


In [ ]:
pip install pillow


In [ ]:
from PIL import Image
import requests
import os
import time

# Your Unsplash Access Key
ACCESS_KEY = "H1IVJhiehu6CL6hTwubD6biPzyvCz8F2u8qW-iadqfI"

# Function to resize and compress images
def resize_and_compress(image_path, output_path, max_size=(800, 800), quality=85):
    """
    Resize and compress an image.
    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the compressed image.
        max_size (tuple): Maximum width and height for resizing.
        quality (int): Compression quality (1-100, higher is better quality).
    """
    with Image.open(image_path) as img:
        img.thumbnail(max_size)  # Resize while maintaining aspect ratio
        img.save(output_path, "JPEG", quality=quality)  # Save as JPEG with compression

# Function to download images from Unsplash
def download_images_unsplash(query, output_dir, num_images=100):
    """
    Download images from Unsplash API.
    Args:
        query (str): Search query for the images.
        output_dir (str): Directory to save the images.
        num_images (int): Total number of images to download.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    url = "https://api.unsplash.com/search/photos"

    headers = {
        "Authorization": f"Client-ID {ACCESS_KEY}"
    }

    total_downloaded = 0
    page = 1

    while total_downloaded < num_images:
        params = {
            "query": query,
            "page": page,
            "per_page": 30  # Unsplash returns a max of 30 images per page
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 403:  # Rate Limit Exceeded
            print("Rate limit reached. Waiting for reset...")
            reset_time = int(response.headers.get("X-Ratelimit-Reset", 60))  # Default 60 seconds
            time.sleep(reset_time)  # Wait for rate limit to reset
            continue
        elif response.status_code != 200:
            print(f"Failed to fetch images: {response.status_code}")
            print("Response:", response.text)
            break

        try:
            data = response.json()  # Attempt to parse the response as JSON
        except requests.exceptions.JSONDecodeError:
            print("Failed to decode JSON from the response.")
            print("Response content:", response.text)
            break

        images = data.get("results", [])

        if not images:
            print("No more images found.")
            break

        for image in images:
            if total_downloaded >= num_images:
                break

            image_url = image["urls"]["full"]  # Full-resolution image URL
            image_id = image["id"]  # Unique image ID

            try:
                img_response = requests.get(image_url, stream=True)

                if img_response.status_code == 200:
                    raw_image_path = os.path.join(output_dir, f"{image_id}_raw.jpg")
                    compressed_image_path = os.path.join(output_dir, f"{image_id}.jpg")

                    # Save the raw image temporarily
                    with open(raw_image_path, "wb") as file:
                        file.write(img_response.content)

                    # Resize and compress the raw image
                    resize_and_compress(raw_image_path, compressed_image_path)

                    # Remove the raw image to save space
                    os.remove(raw_image_path)

                    total_downloaded += 1
                    print(f"Downloaded and compressed {total_downloaded}/{num_images}: {compressed_image_path}")
                else:
                    print(f"Failed to download image {image_id}")
            except Exception as e:
                print(f"Error downloading image {image_id}: {e}")

        page += 1  # Move to the next page
        time.sleep(1)  # Add delay between pages to avoid hitting rate limits

# Dataset with species and subspecies
dataset = {
    #"Bald eagle": ["Bald eagle"],
    "Harpy Eagle": ["Harpy Eagle"],
    #"Golden eagle": ["Golden Eagle"],
    #"Polar Bear": ["Polar Bear"],
    #"Grizzly bear": ["Grizzly Bear"],
    #"Siberian Tiger": ["Siberian Tiger"],
    #"Sumatran Tiger": ["Sumatran Tiger"],
    #"African elephant": ["African Elephant"],
    #"Asian elephant": ["Asian Elephant"],


    # Add more species and subspecies here
}

# Base directory for saving images
output_dir_base = "/content/drive/MyDrive/Unsplash_Images"

# Number of images per query
num_images = 700  # Change this to the total number of images you want per query

# Download images for each species and subspecies
for species, subspecies_list in dataset.items():
    for subspecies in subspecies_list:
        output_dir = os.path.join(output_dir_base, species, subspecies.replace(" ", "_"))
        download_images_unsplash(subspecies, output_dir, num_images)


In [ ]:
import os

def count_images_in_dataset(base_dir):
    """
    Count the number of images for each species and subspecies in the dataset directory.

    Args:
        base_dir (str): Base directory containing the dataset.

    Returns:
        dict: A nested dictionary with species as keys and subspecies counts as values.
    """
    image_counts = {}

    for species in os.listdir(base_dir):
        species_dir = os.path.join(base_dir, species)
        if not os.path.isdir(species_dir):
            continue

        species_counts = {}

        for subspecies in os.listdir(species_dir):
            subspecies_dir = os.path.join(species_dir, subspecies)
            if not os.path.isdir(subspecies_dir):
                continue

            # Count .jpg files in the subspecies directory
            image_count = sum(
                1 for file in os.listdir(subspecies_dir)
                if file.lower().endswith((".jpg", ".jpeg"))
            )
            species_counts[subspecies] = image_count

        image_counts[species] = species_counts

    return image_counts

# Base directory for the dataset
output_dir_base = "/content/drive/MyDrive/Unsplash_Images"

# Count images in the dataset
image_counts = count_images_in_dataset(output_dir_base)

# Display the counts
for species, subspecies_counts in image_counts.items():
    print(f"\nSpecies: {species}")
    for subspecies, count in subspecies_counts.items():
        print(f"  Subspecies: {subspecies.replace('_', ' ')} - {count} images")



Species: Polar Bear
  Subspecies: Polar Bear - 399 images

Species: Siberian Tiger
  Subspecies: Siberian Tiger - 828 images

Species: Golden eagle
  Subspecies: Golden Eagle - 119 images

Species: Grizzly bear
  Subspecies: Grizzly Bear - 350 images

Species: Sumatran Tiger
  Subspecies: Sumatran Tiger - 682 images

Species: African elephant
  Subspecies: African Elephant - 607 images

Species: Asian elephant
  Subspecies: Asian Elephant - 546 images

Species: Bald eagle
  Subspecies: Bald eagle - 627 images


In [ ]:
import os
import random
import shutil

def split_dataset(input_dir, output_dir, train_percent=0.7, val_percent=0.15, test_percent=0.15):
    """
    Split dataset into training, validation, and test subsets.

    Args:
        input_dir (str): The directory containing the original dataset.
        output_dir (str): The directory where the split dataset will be saved.
        train_percent (float): The percentage of images to be used for training.
        val_percent (float): The percentage of images to be used for validation.
        test_percent (float): The percentage of images to be used for testing.
    """
    # Ensure the output directories exist
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Iterate through each species directory
    for species in os.listdir(input_dir):
        species_dir = os.path.join(input_dir, species)

        if not os.path.isdir(species_dir):
            continue

        # Create corresponding directories for train, val, and test sets
        os.makedirs(os.path.join(train_dir, species), exist_ok=True)
        os.makedirs(os.path.join(val_dir, species), exist_ok=True)
        os.makedirs(os.path.join(test_dir, species), exist_ok=True)

        # Iterate through each subspecies directory under species
        for subspecies in os.listdir(species_dir):
            subspecies_dir = os.path.join(species_dir, subspecies)

            if not os.path.isdir(subspecies_dir):
                continue

            # List all image files in the subspecies directory
            images = [f for f in os.listdir(subspecies_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            random.shuffle(images)  # Shuffle the images randomly

            num_images = len(images)
            num_train = int(train_percent * num_images)
            num_val = int(val_percent * num_images)
            num_test = num_images - num_train - num_val

            # Split images into train, val, and test
            train_images = images[:num_train]
            val_images = images[num_train:num_train+num_val]
            test_images = images[num_train+num_val:]

            # Copy the images into corresponding directories
            for img in train_images:
                shutil.copy(os.path.join(subspecies_dir, img), os.path.join(train_dir, species, img))
            for img in val_images:
                shutil.copy(os.path.join(subspecies_dir, img), os.path.join(val_dir, species, img))
            for img in test_images:
                shutil.copy(os.path.join(subspecies_dir, img), os.path.join(test_dir, species, img))

    print("Dataset splitting complete!")


# Example usage:
input_dir = "/content/drive/MyDrive/Unsplash_Images"  # Directory where your dataset is stored
output_dir = "/content/drive/MyDrive/Unsplash_Images_Split"  # Directory where the split dataset will be saved

# Split the dataset into 70% train, 15% validation, and 15% test
split_dataset(input_dir, output_dir, train_percent=0.7, val_percent=0.15, test_percent=0.15)


Dataset splitting complete!


The code below deletes the directory (files and subdirectories) permanently.

In [ ]:
import shutil

shutil.rmtree('/content/drive/MyDrive/Unsplash_Images/Harpy Eagle')